In [1]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

## SITEMAPREADER

In [170]:
from llama_index.readers.web import SitemapReader

#sitemap="https://hacemosescuela.cba.gov.ar/post-sitemap.xml"
sitemap="https://hacemosescuela.cba.gov.ar/page-sitemap.xml"

documents = SitemapReader(html_to_text=True, limit=10).load_data(
    sitemap_url=sitemap
)

error fetching page from https://hacemosescuela.cba.gov.ar/secuencias/


In [171]:
len(documents)

37

In [3]:
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch the web page
url = "https://hacemosescuela.cba.gov.ar/materiales-educativos/"
response = requests.get(url)

if response.status_code == 200:
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Step 3: Extract metadata
    meta_tags = soup.find_all('meta')
    metadata = {}
    for tag in meta_tags:
        if 'name' in tag.attrs:
            name = tag.attrs['name']
            content = tag.attrs.get('content', '')
            metadata[name] = content
        elif 'property' in tag.attrs:  # For OpenGraph metadata
            property = tag.attrs['property']
            content = tag.attrs.get('content', '')
            metadata[property] = content

    # Display the metadata
    for key, value in metadata.items():
        print(f"{key}: {value}")
else:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")
    


viewport: width=device-width, initial-scale=1
robots: index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1
og:locale: es_ES
og:type: article
og:title: Materiales educativos - Hacemos escuela
og:description: Materiales educativos En este espacio encontrarán materiales ordenados por nivel educativo. Fueron diseñados para diferentes escenarios escolares: presencial, no presencial o combinado. Tienen el propósito de ofrecer desarrollos didácticos para acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de los docentes de todos los niveles. Estos materiales se inscriben en los […]
og:url: https://hacemosescuela.cba.gov.ar/materiales-educativos/
og:site_name: Hacemos escuela
article:modified_time: 2023-05-30T14:29:38+00:00
og:image: https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-materiales-educativos-1024x267.jpg
twitter:card: summary_large_image
twitter:label1: Tiempo de lectura
twitter:data1: 6 minutos
generator: Elementor 

In [4]:
def get_metadata_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Step 2: Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Step 3: Extract metadata
        meta_tags = soup.find_all('meta')
        metadata = {}
        for tag in meta_tags:
            if 'name' in tag.attrs:
                name = tag.attrs['name']
                content = tag.attrs.get('content', '')
                metadata[name] = content
            elif 'property' in tag.attrs:  # For OpenGraph metadata
                property = tag.attrs['property']
                content = tag.attrs.get('content', '')
                metadata[property] = content
        
        return metadata

In [5]:
get_metadata_from_url(url)

{'viewport': 'width=device-width, initial-scale=1',
 'robots': 'index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1',
 'og:locale': 'es_ES',
 'og:type': 'article',
 'og:title': 'Materiales educativos - Hacemos escuela',
 'og:description': 'Materiales educativos En este espacio encontrarán materiales ordenados por nivel educativo. Fueron diseñados para diferentes escenarios escolares: presencial, no presencial o combinado. Tienen el propósito de ofrecer desarrollos didácticos para acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de los docentes de todos los niveles. Estos materiales se inscriben en los […]',
 'og:url': 'https://hacemosescuela.cba.gov.ar/materiales-educativos/',
 'og:site_name': 'Hacemos escuela',
 'article:modified_time': '2023-05-30T14:29:38+00:00',
 'og:image': 'https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-materiales-educativos-1024x267.jpg',
 'twitter:card': 'summary_large_image',
 'twitter:labe

# Probando LlamaIndex para sitios web

In [172]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [173]:
## Configuración para debugging

from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [174]:
from llama_index.core import Settings

Settings.callback_manager = callback_manager

#Settings.chunk_overlap = 0

In [15]:
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True,
                                metadata_fn=get_metadata_from_url).load_data(
    ["https://hacemosescuela.cba.gov.ar/materiales-educativos/"]
)

documents[0]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
DEBUG:urllib3.connectionpool:https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
DEBUG:urllib3.connectionpool:https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None


Document(id_='https://hacemosescuela.cba.gov.ar/materiales-educativos/', embedding=None, metadata={'viewport': 'width=device-width, initial-scale=1', 'robots': 'index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1', 'og:locale': 'es_ES', 'og:type': 'article', 'og:title': 'Materiales educativos - Hacemos escuela', 'og:description': 'Materiales educativos En este espacio encontrarán materiales ordenados por nivel educativo. Fueron diseñados para diferentes escenarios escolares: presencial, no presencial o combinado. Tienen el propósito de ofrecer desarrollos didácticos para acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de los docentes de todos los niveles. Estos materiales se inscriben en los […]', 'og:url': 'https://hacemosescuela.cba.gov.ar/materiales-educativos/', 'og:site_name': 'Hacemos escuela', 'article:modified_time': '2023-05-30T14:29:38+00:00', 'og:image': 'https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-ma

In [175]:
len(documents)

37

In [75]:
print(documents[20].get_text())

[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_3-1/img/logo_202403.png?v=20240523-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
 

In [74]:
print(documents[3].get_metadata_str())

Source: https://hacemosescuela.cba.gov.ar/buscador/


In [76]:
from llama_index.core.node_parser import HTMLNodeParser

parser = HTMLNodeParser()  # optional list of tags
nodes = parser.get_nodes_from_documents(documents)

In [77]:
len(nodes)

0

## Estrategia Markdown

In [176]:
## Pruebo un parser para markdown

from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser.from_defaults(include_metadata=True,
                                               include_prev_next_rel=True)

nodes = node_parser.get_nodes_from_documents(documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Programas

[ ![entre-lenguas](https://hacemoses...
> Adding chunk: Programas

[ ![entre-lenguas](https://hacemoses...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Otros sitios de interés

[ ![boton-rdu](https:/...
> Adding chunk: Otros sitios de interés

[ ![boton-rdu](https:/...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: **hacemosescuela@isep-cba.edu.ar**

[ ![](https...
> Adding chunk: **hacemosescuela@isep-cba.edu.ar**

[ ![](https...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: **¡Seguinos!**

](https://www.instagram.com/Hac...
> Adding chunk: **¡Seguinos!**

](https://www.instagram.com/Hac...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.g

In [80]:
len(nodes)

310

In [81]:
type(nodes)

list

In [177]:
for i, n in enumerate(nodes):
    if len(n.get_content()) > 10000:
        print(len(n.get_content()))
        print(i)
        print("===============")

11304
0


In [178]:
nodes_small = [n for n in nodes if len(n.get_content()) < 10000]

In [251]:
len(nodes_small)

309

In [180]:
nodes_small2 = [n for n in nodes_small if len(n.metadata) > 1]

In [252]:
len(nodes_small2)

273

In [182]:
# Agregamos información de metadata  cada nodo! (TEST)
for n in nodes_small2:
    n.set_content(n.get_metadata_str() + '\n' + n.get_content())

In [250]:
print(nodes_small2[90].get_content())

Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-ficciones-
autobiograficas/)

[Revoluciones del yo: ficciones
autobiográficas](https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-
ficciones-autobiograficas/)

“Revoluciones del yo: ficciones autobiográficas” es una secuencia didáctica de
Hacemos Escuela que ofrece un recorrido por...

[](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-recorrido-
por-america-latina/)

[¡Música y revoluciones! Un breve recorrido por América
Latina](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-
recorrido-por-america-latina/)

"¡Música y revoluciones! Un breve recorrido por América Latina" es una
secuencia didáctica de Hacemos Escuela...

[](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-ritornelo-de-la-
memoria-historica/)

[Música y revoluciones! Ritornelo de la memoria
histórica](ht

In [154]:
print(nodes_small2[100].get_content())

**Narrar las prácticas, reconstruir experiencias**

_Conversar con un docente, o con un grupo de docentes,  
_ _es una invitación a escuchar historias de escuela.  
_ _Es un convite a detenerse y sumergirse en relatos en primera persona  
_ _que narran experiencias escolares  
_ _y que confiesan las sutiles percepciones._

 _(McEwan, citado en Suárez y otros, 2021)_

En este espacio, compartimos una serie de conversaciones entre docentes de los
niveles Inicial, Primario y Secundario de diferentes localidades de la
provincia de Córdoba en el marco de la Comunidad de prácticas: la clase en
plural perteneciente a la plataforma Tu Escuela en Casa. La iniciativa parte
de reconocer la importancia de promover un diálogo acerca de la enseñanza, un
espacio de escucha y encuentro entre colegas en donde compartir y reflexionar
en torno a experiencias pedagógicas particulares y situadas.

Creemos que documentar las formas que toma la enseñanza en una institución
permite reconstruir prácticas que s

In [147]:
for i,n in enumerate(nodes_small2):
    if n.metadata["Source"].startswith("https://hacemosescuela.cba.gov.ar/materiales-educativos/"):
        print(i)
        print("=====================")

87
88
89
90
91
92


In [254]:
print(nodes_small2[90].metadata)

{'Header_2': 'Educación Secundaria', 'Source': 'https://hacemosescuela.cba.gov.ar/materiales-educativos/'}


In [90]:
print(documents[0].metadata)

{'Source': 'https://hacemosescuela.cba.gov.ar/'}


In [255]:
from llama_index.core.schema import MetadataMode

print(nodes_small2[90].get_content(metadata_mode=MetadataMode.LLM))

Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/

Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-ficciones-
autobiograficas/)

[Revoluciones del yo: ficciones
autobiográficas](https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-
ficciones-autobiograficas/)

“Revoluciones del yo: ficciones autobiográficas” es una secuencia didáctica de
Hacemos Escuela que ofrece un recorrido por...

[](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-recorrido-
por-america-latina/)

[¡Música y revoluciones! Un breve recorrido por América
Latina](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-
recorrido-por-america-latina/)

"¡Música y revoluciones! Un breve recorrido por América Latina" es una
secuencia didáctica de Hacemos Escuela...

[](https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-ri

In [106]:
print(nodes_small[30].get_content())

Educación Secundaria / Secundaria común / Ciclo Básico / Ciencias
Sociales / Historia / Geografía /

[patria](/hacemosescuela/buscador/?key=patria)
[revolución](/hacemosescuela/buscador/?key=revolución)
[efemérides](/hacemosescuela/buscador/?key=efemérides) [25 de
Mayo](/hacemosescuela/buscador/?key=25 de Mayo)
[mujeres](/hacemosescuela/buscador/?key=mujeres)
[multiperspectividad](/hacemosescuela/buscador/?key=multiperspectividad)
[estados nacionales](/hacemosescuela/buscador/?key=estados nacionales)
[multicausalidad](/hacemosescuela/buscador/?key=multicausalidad)
[fuentes](/hacemosescuela/buscador/?key=fuentes) [maría
remedios](/hacemosescuela/buscador/?key=maría remedios) [estratificación
social](/hacemosescuela/buscador/?key=estratificación social) [orden
colonial](/hacemosescuela/buscador/?key=orden colonial)

[**Acceder**](https://hacemosescuela.cba.gov.ar/25-de-mayo-protagonistas-de-
una-revolucion/)

[ ![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2023/05/primaria-
so

In [22]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

### Retriever

In [256]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

#model_name = "models/embedding-001"
model_name = "models/text-embedding-004"

embed_model = GeminiEmbedding(
    model_name=model_name, 
    api_key=GOOGLE_API_KEY,
    embed_batch_size=20
)

In [257]:
## PROBAMOS OTRA BASE DE DATOS VECTORIAL!


from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

# creates a persistant index to disk
#client = QdrantClient(host="localhost", port=6333)
#aclient = AsyncQdrantClient(host="localhost", port=6333)

client = QdrantClient(":memory:")

# create our vector store with hybrid indexing enabled
# batch_size controls how many nodes are encoded with sparse vectors at once
vector_store = QdrantVectorStore(
    "hacemos_escuela_isep",
    client=client,
    #aclient=aclient,
    enable_hybrid=True,
    batch_size=12,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
#Settings.chunk_size = 1500



DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293
https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293
https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [259]:
Settings.chunk_size = 1500
Settings.chunk_overlap = 300

In [ ]:
# Probamos otro embedding por las dudas haya una limitante en el servicio de embeddings de Google

#from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
# jinaai/jina-embeddings-v2-base-es
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

### Index

*IMPORTANTE*: Aparentemente para nodos con una logitud mayor a 10000 (de contenido) el servicio no funciona (tiene un límite)!!

In [260]:
# demora 7 min aprox para 300 nodos 
from llama_index.core import VectorStoreIndex


base_index = VectorStoreIndex(
    nodes=nodes_small2,
    embed_model=embed_model,
    storage_context=storage_context)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 12.183147 seconds
    |_CBEventType.EMBEDDING -> 6.399676 seconds
    |_CBEventType.EMBEDDING -> 6.578037 seconds
    |_CBEventType.EMBEDDING -> 6.39466 seconds
    |_CBEventType.EMBEDDING -> 6.514601 seconds
    |_CBEventType.EMBEDDING -> 6.442709 seconds
    |_CBEventType.EMBEDDING -> 6.379365 seconds
    |_CBEventType.EMBEDDING -> 6.39452 seconds
    |_CBEventType.EMBEDDING -> 6.52552 seconds
    |_CBEventType.EMBEDDING -> 6.394936 seconds
    |_CBEventType.EMBEDDING -> 6.536223 seconds
    |_CBEventType.EMBEDDING -> 6.446187 seconds
    |_CBEventType.EMBEDDING -> 6.404548 seconds
    |_CBEventType.EMBEDDING -> 4.12004 seconds
**********


In [261]:
# Almacenamos index localmente para evitar que se procese nuevamente!
base_index.storage_context.persist(persist_dir="data/")

DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/docstore.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/docstore.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/index_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/index_store.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/graph_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/graph_store.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/image__vector_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/image__vector_store.json


In [ ]:
# Cargamos el index!!

from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context_reb = StorageContext.from_defaults(persist_dir="data/")

# load index
index_reb = load_index_from_storage(storage_context,
                                embed_model=embed_model)

In [262]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model_name="models/gemini-1.0-pro", temperature=0.3)

In [263]:
print(llm.model_name)

models/gemini-1.0-pro


In [264]:
from llama_index.core import PromptTemplate


qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un asistente y debe responder a la pregunta del usuario usando el contexto provisto."
    "Responda de forma amable y detallada, proporcionando toda la información necesaria."
    "Si es posible organice y estructure la información mediante inferencia sobre el contexto provisto."
    "Además responda con enlaces a los recursos en caso de que sea posible."
    "Si no encuentra la respuesta en el contexto pero el mismo contiene información y enlaces relacionados a la pregunta, responda con esa información."
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

qa_template = PromptTemplate(qa_prompt_str)


In [268]:
retriever = base_index.as_retriever(similarity_top_k=5)

nodes_with_scores = retriever.retrieve("Qué material educativo hay para educación inicial?")

for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

DEBUG:llama_index.core.indices.utils:> Top 5 nodes:
> [Node 33aa56fb-c416-408b-91db-be20ae18541e] [Similarity score:             0.857357] Header_2: EDUCACIÓN PRIMARIA · PRIMER CICLO  
Source: https://hacemosescuela.cba.gov.ar/cultura-d...
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.855923] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
...
> [Node 4423f1e0-87f1-48cd-aa56-fb42062756a9] [Similarity score:             0.847663] Header_2: EDUCACIÓN PRIMARIA · PRIMER CICLO  
Source: https://hacemosescuela.cba.gov.ar/cultura-d...
> [Node 6e60eb10-a9c2-49bc-9700-51362d8d8d5c] [Similarity score:             0.830292] Header_2: EDUCACIÓN PRIMARIA · PRIMER CICLO  
Source: https://hacemosescuela.cba.gov.ar/cultura-d...
> [Node 067f42ce-d47e-420a-97a4-81e7c982d59b] [Similarity score:             0.829886] Header_2:  **Colecciones Didácticas
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://...
> Top 5 

In [269]:
index_sel = base_index

query_engine = index_sel.as_query_engine(
    llm=llm,
    similarity_top_k=5,
    sparse_top_k=10,
    vector_store_query_mode="hybrid",
    text_qa_template=qa_template
)

In [274]:
from llama_index.core.response.notebook_utils import display_response

response = query_engine.query(
    "qué material educativo hay para la educación secundaria?"
)

display_response(response)

DEBUG:llama_index.core.indices.utils:> Top 5 nodes:
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.5] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
...
> [Node b2d57825-eeec-4409-88d2-4559c0aea9e4] [Similarity score:             0.5] Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
E...
> [Node 78c38642-28f0-41e0-bf77-3ed2fc792297] [Similarity score:             0.405156] Header_2: Educación Secundaria
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacem...
> [Node c530cae5-e2bc-4d86-9a83-4b35d3ced0bf] [Similarity score:             0.257287] Header_2: Educación Secundaria
Header_6: **¡Seguinos!**
Source: https://hacemosescuela.cba.gov.ar...
> [Node 1068da60-5faf-43d1-8867-0f1ab6ccff26] [Similarity score:             0.248585] Header_3:  **Educación Domiciliaria y Hospitalaria
Source: https://hacemosescuela.cba.gov.ar/educ...
> Top 5 nodes:
> [

**`Final Response:`** **Materiales educativos para la Educación Secundaria:**

* **Revoluciones del yo: ficciones autobiográficas:** https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-ficciones-autobiograficas/
* **¡Música y revoluciones! Un breve recorrido por América Latina:** https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-recorrido-por-america-latina/
* **¡Música y revoluciones! Ritornelo de la memoria histórica:** https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-ritornelo-de-la-memoria-historica/
* **Recomendación didáctica. Enfoques en el plano: documentos y aportes para la enseñanza de Artes Visuales en las escuelas:** https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-escuelas/
* **Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos, reivindicaciones y luchas:** https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/
* **Efemérides: 25 de Mayo. Patria, revolución y guerra:** https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-y-guerra/
* **25 de Mayo: protagonistas de una revolución:** https://hacemosescuela.cba.gov.ar/25-de-mayo-protagonistas-de-una-revolucion/
* **La revolución de las lenguas y los lectos:** https://hacemosescuela.cba.gov.ar/la-revolucion-de-las-lenguas-y-los-lectos-2/
* **La probabilidad. La revolución de los datos:** https://hacemosescuela.cba.gov.ar/la-probabilidad-la-revolucion-de-los-datos/
* **Recomendación didáctica: mundos pluriversos:** https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-pluriversos/
* **Unidades de longitud: el metro. La revolución de las medidas:** https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-revolucion-de-las-medidas/
* **La revolución de los microorganismos (Parte II):** https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-parte-ii/

---

In [379]:
QUERY_GEN_PROMPT = (
    "Usted es un asistente en un contexto educativo que genera múltiples consultas o preguntas en base a "
    "úna pregunta inicial. Intente reemplazar términos o palabras con sinónimos."
    "Por ejemplo si se solicita información para un 'nivel' educativo específico, puede generar sinónimos"
    "del término 'nivel' usando el término 'educación', por ejemplo reemplazar 'nivel inicial' por 'educación inicial'"
    "Genere {num_queries} preguntas de búsqueda, una en cada línea,"
    "relacionadas a la siguiente pregunta inicial:\n"
    "Pregunta: {query}\n"
    "Agregue la pregunta original en el listado de preguntas generadas y si es posible corrija posibles errores o typos."
    "Preguntas:\n"
)

In [385]:
from llama_index.core.retrievers import QueryFusionRetriever

vector_retriever = base_index.as_retriever(
    vector_store_query_mode="default",
    similarity_top_k=8,
)

text_retriever = base_index.as_retriever(
    vector_store_query_mode="sparse",
    similarity_top_k=8,  # interchangeable with sparse_top_k in this context
    qa_prompt_str=qa_prompt_str
)

retriever = QueryFusionRetriever(
    [vector_retriever, text_retriever],
    llm=llm,
    similarity_top_k=10,
    num_queries=5 ,  # set this to 1 to disable query generation
    #mode="reciprocal_rerank",
    mode="relative_score", # ESTO INFLUYE EN EL USO DE QDRANT?
    use_async=False, # ESTO INFLUYE EN EL USO DE QDRANT?
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT,  # we could override the query generation prompt here
)

In [386]:
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.query_engine import RetrieverQueryEngine


#response_synthesizer = CompactAndRefine()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    #response_synthesizer=response_synthesizer,
).from_args(retriever=retriever, text_qa_template=qa_template)

In [390]:
response = query_engine.query(
    "necesito secuencias didácticas sobre programación"
)

display_response(response)

Generated queries:
1. Secuencias pedagógicas para la enseñanza de la codificación
2. Estrategias de instrucción para la educación en programación
3. Planes de lecciones para el aprendizaje de la programación
4. Necesito secuencias didácticas sobre programación
DEBUG:llama_index.core.indices.utils:> Top 8 nodes:
> [Node 7ea7aa42-fb48-47a0-9be0-753c4d63c66f] [Similarity score:             0.881329] Header_2: EJE: Algoritmos y programación
Source: https://hacemosescuela.cba.gov.ar/cultura-digita...
> [Node 444e0470-6d81-4af3-86b1-2c5e248f84a2] [Similarity score:             0.87606] Header_2: Presentación
Header_4: **Clase 1**
Source: https://hacemosescuela.cba.gov.ar/cultura-di...
> [Node 193ad854-6974-486e-a4f2-bc7c5a9f826a] [Similarity score:             0.872881] Header_2: Presentación
Header_4: **Clase 3**
Source: https://hacemosescuela.cba.gov.ar/cultura-di...
> [Node 38002ff5-a9c6-487b-8418-5690c38febb3] [Similarity score:             0.872264] Header_2: Presentación
Header_4: **Cl

**`Final Response:`** **Secuencias didácticas sobre programación:**

* **Nivel Primario / Primer Ciclo - Cultura Digital en la Quinta Hora**
    * [Eje Sistemas Digitales de la Información](https://hacemosescuela.cba.gov.ar/culturadigital/eje-sistemas-digitales-de-la-informacion/)
    * [Eje Algoritmos y Programación](https://hacemosescuela.cba.gov.ar/culturadigital/eje-algoritmos-y-programacion/)
    * [Eje Ciudadanía](https://hacemosescuela.cba.gov.ar/culturadigital/eje-ciudadania/)

* **Nivel Primario /Segundo Ciclo - Educación Tecnológica**
    * [Eje Sistemas Digitales de la Información](https://hacemosescuela.cba.gov.ar/culturadigital/eje-sistemas-digitales-de-la-informacion-nivel-primario/)
    * [Eje Algoritmos y Programación](https://hacemosescuela.cba.gov.ar/culturadigital/eje-algoritmos-y-programacion-nivel-primario/)
    * [Eje Ciudadanía](https://hacemosescuela.cba.gov.ar/culturadigital/eje-ciudadania-nivel-primario/)

* **Nivel Secundario / Ciclo Básico - Educación Tecnológica**
    * [Eje Sistemas Digitales de la Información](https://hacemosescuela.cba.gov.ar/culturadigital/eje-sistemas-digitales-de-la-informacion-nivel-secundario/)
    * [Eje Algoritmos y Programación](https://hacemosescuela.cba.gov.ar/culturadigital/eje-algoritmos-y-programacion-nivel-secundario/)
    * [Eje Ciudadanía](https://hacemosescuela.cba.gov.ar/culturadigital/eje-ciudadania-nivel-secundario/)

In [62]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [283]:
nodes_with_scores = retriever.retrieve(
    "qué material educativo hay para el nivel secundario?"
)

DEBUG:google.api_core.retry:Retrying due to 503 Connection timed out, sleeping 0.4s ...
Retrying due to 503 Connection timed out, sleeping 0.4s ...
Generated queries:
- ¿Qué recursos educativos están disponibles para estudiantes de secundaria?
- ¿Qué materiales de aprendizaje son adecuados para el nivel de educación secundaria?
- ¿Qué material educativo hay para el nivel secundario?
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.865302] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
...
> [Node 78c38642-28f0-41e0-bf77-3ed2fc792297] [Similarity score:             0.85163] Header_2: Educación Secundaria
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacem...
> Top 2 nodes:
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.865302] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-

In [121]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

Score: 0.07 - **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-9/img/ig.png)...
-----

Score: 0.07 - **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-9/img/licencias.png)

![Logo Ministerio de Educación](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-9/img/Logo_MIN.png)![Logo Gobierno de
Córdoba](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-9/img/Logo_GOB.png)...
-----



### Query Engine

In [78]:
qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un asistente y debe responder a la pregunta del usuario usando el contexto provisto."
    "Responda de forma amable y detallada, proporcionando toda la información necesaria."
    "Además responda con enlaces a los recursos en caso de que sea posible."
    "Si no encuentra la respuesta en el contexto pero el mismo contiene información y enlaces al respecto, responda con esa información."
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

In [284]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            ""
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [285]:
from llama_index.core.query_engine import RetrieverQueryEngine



query_engine = RetrieverQueryEngine.from_args(retriever, 
                                              llm=llm, 
                                              text_qa_template=text_qa_template)

In [288]:
response = query_engine.query("¿Qué material hay para la educación secundaria?")

Generated queries:
- ¿Qué recursos educativos están disponibles para la educación secundaria?
- ¿Dónde puedo encontrar materiales de enseñanza para la educación secundaria?
- ¿Qué material hay para la educación secundaria?
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 78c38642-28f0-41e0-bf77-3ed2fc792297] [Similarity score:             0.881147] Header_2: Educación Secundaria
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacem...
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.87061] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
...
> Top 2 nodes:
> [Node 78c38642-28f0-41e0-bf77-3ed2fc792297] [Similarity score:             0.881147] Header_2: Educación Secundaria
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacem...
> [Node 843168b9-e568-48b1-866f-771ea37e1964] [Similarity score:             0.87061] Header_2: Materiales educativos
Source: https://hacemosescuel

In [289]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** El sitio web "Hacemos Escuela" ofrece materiales educativos para la educación secundaria, diseñados para diferentes escenarios escolares (presencial, no presencial o combinado). Estos materiales tienen como objetivo apoyar la escolaridad y ayudar a los docentes a diseñar propuestas de enseñanza.

Los materiales se basan en los contenidos y aprendizajes priorizados por los programas del Ministerio de Educación.

Puedes encontrar estos materiales en el siguiente enlace:

https://hacemosescuela.cba.gov.ar/materiales-educativos/

---

In [ ]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2500, 1800, 900])
nodes = node_parser.get_nodes_from_documents(documents)
len(nodes)

In [10]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [11]:
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

7

In [12]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

7

In [13]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [14]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext


docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore, )


In [15]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="this is a document"
)

In [16]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model
)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 3.242697 seconds
**********


In [17]:
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = base_index.as_retriever(similarity_top_k=4)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [33]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "educación secundaria"
)

nodes_ret = retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 4 nodes:
> [Node 3bade23b-76df-4004-b9f3-cfed18c47f8c] [Similarity score:             0.85462] **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/ha...
> [Node 70ce5d21-826e-4203-b344-4172fe4bc4bd] [Similarity score:             0.840263] **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ ...
> [Node bd661889-f4fa-4f84-990f-ee7752b55bee] [Similarity score:             0.789928] Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fuer...
> [Node 83cca01d-50c6-4e11-b7a5-09b3bd45c18a] [Similarity score:             0.76154] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> Top 4 nodes:
> [Node 3bade23b-76df-4004-b9f3-cfed18c47f8c] [Similarity score:             0.85462] **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/ha

In [17]:
base_nodes = base_retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 6341246f-da8f-4c9e-8d17-979150616839] [Similarity score:             0.7359] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.725763] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.724245] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 5f91ae0c-c714-46a1-8ff1-6249778c0257] [Similarity score:             0.714791] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.709381] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 23



---



In [19]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes_ret:
    display_source_node(node, source_length=10000)

DEBUG:matplotlib:matplotlib data path: /home/illak/Proyectos/web_RAG_pruebas/.conda/lib/python3.11/site-packages/matplotlib/mpl-data
matplotlib data path: /home/illak/Proyectos/web_RAG_pruebas/.conda/lib/python3.11/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/illak/.config/matplotlib
CONFIGDIR=/home/illak/.config/matplotlib
DEBUG:matplotlib:interactive is False
interactive is False
DEBUG:matplotlib:platform is linux
platform is linux
DEBUG:matplotlib:CACHEDIR=/home/illak/.cache/matplotlib
CACHEDIR=/home/illak/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/illak/.cache/matplotlib/fontlist-v390.json
Using fontManager instance from /home/illak/.cache/matplotlib/fontlist-v390.json


**Node ID:** 3bade23b-76df-4004-b9f3-cfed18c47f8c<br>**Similarity:** 0.8546203119744958<br>**Text:** **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/ig.png)<br>

**Node ID:** 70ce5d21-826e-4203-b344-4172fe4bc4bd<br>**Similarity:** 0.8402628922427792<br>**Text:** **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/licencias.png)

![Logo Footer](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_footer_2024.png)<br>

**Node ID:** bd661889-f4fa-4f84-990f-ee7752b55bee<br>**Similarity:** 0.7899284603700811<br>**Text:** Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.<br>

**Node ID:** 83cca01d-50c6-4e11-b7a5-09b3bd45c18a<br>**Similarity:** 0.7615396796885139<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##<br>

In [19]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** 6341246f-da8f-4c9e-8d17-979150616839<br>**Similarity:** 0.7359004959511858<br>**Text:** La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-lo-largo-
de-la-escolaridad/)

[La enseñanza de los números a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-
lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que brinda materiales
sobre contenidos vinculados con la enseñanza...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-a-lo-
largo-de-la-escolaridad/)

[La enseñanza de la descripción a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-
a-lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que ofrece materiales
sobre las prácticas del lenguaje vinculadas...

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-biodiversidad-en-
la-provincia-de-cordoba/)

[Recomendación didáctica: Biodiversidad en la Provincia de
Córdoba](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-
biodiversidad-en-la-provincia-de-cordoba/)

“Biodiversidad en la provincia de Córdoba” es una recomendación didáctica de
Hacemos Escuela destinada al Primer y Segundo...

[](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-asombrosos-
fotomontajes-ludicos/)

[Fotografía en el aula. Paseos asombrosos, fotomontajes
lúdicos](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-
asombrosos-fotomontajes-ludicos/)

“Fotografía en el aula. Paseos asombrosos, fotomontajes lúdicos” es una
secuencia didáctica de Hacemos Escuela destinada...

[](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-tu-mesa/)

[De la colmena a tu mesa](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-
tu-mesa/)

"De la colmena a tu mesa" es una secuencia didáctica de Tu Escuela en Casa
destinada a 1.º grado de nivel Primario....

[ __ Ver más materiales del nivel Primario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=161&key=)

## Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica.<br>

**Node ID:** 8415fd8f-8321-4cb0-b64f-544be7b2c894<br>**Similarity:** 0.7257628669636027<br>**Text:** Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa...

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-iii/)

[Un recorrido por el jardín (Parte III)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-iii/)

“Un recorrido por el jardín (Parte III)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-ii/)

[Un recorrido por el jardín (Parte II)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-ii/)

“Un recorrido por el jardín (Parte II)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-i/)

[Un recorrido por el jardín (Parte I)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-i/)

“Un recorrido por el jardín (Parte I)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

[En la red, pero sin enredarme (Parte
II)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

"En la red, pero sin enredarme (Parte II)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

[En la red, pero sin enredarme (Parte
I)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

"En la red, pero sin enredarme (Parte I)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

[Había una vez… ¡te lo cuento otra
vez!](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

Había una vez… ¡te lo cuento otra vez! es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica.<br>

**Node ID:** 1d80ac65-2cbf-451c-ad82-ff04ff364c20<br>**Similarity:** 0.7242450938752326<br>**Text:** La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

[La revolución de los microorganismos (Parte
II)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

“La revolución de los microorganismos (Parte II)” es una secuencia didáctica
de Hacemos Escuela para 1.° y 2.° año del nivel...

[](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-desafian-el-
aula-estandar/)

[Experiencias escolares que desafían el aula
estándar](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-
desafian-el-aula-estandar/)

Objeto hipermedial interactivo (OHI) diseñado en el marco de la Actualización
Académica dirigida al Nivel de Educación Superior:...

[](https://hacemosescuela.cba.gov.ar/biblioteca-pedagogica/)

[Biblioteca pedagógica](https://hacemosescuela.cba.gov.ar/biblioteca-
pedagogica/)

Objeto hipermedial interactivo (OHI) que brinda una amplia selección de libros
y películas recopilados en una biblioteca...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

[La revolución de los microorganismos (Parte
I)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

“La revolución de los microorganismos (Parte I)” es una secuencia didáctica de
Hacemos Escuela para 1.° y 2.° año del nivel...

[ __ Ver más materiales del nivel Secundario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=165&key=)

![ISEP](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/isep-footer.png)

Los contenidos que se ponen a disposición en esta plataforma son creados y
curados por el Instituto Superior de Estudios Pedagógicos (ISEP), con el
aporte en la producción de los equipos técnicos de las diferentes Direcciones
Generales del Ministerio de Educación de la provincia de Córdoba.

![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/correo-footer.png)

###### **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/ig.png)

###### **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/licencias.png)

![Logo Footer](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_footer_2024.png)<br>

**Node ID:** 5f91ae0c-c714-46a1-8ff1-6249778c0257<br>**Similarity:** 0.7147914215776487<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##

## Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.

## Educación Inicial

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

**Node ID:** a8a05707-7c0b-4894-98e3-f083557556b0<br>**Similarity:** 0.7093812976063829<br>**Text:** Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-
en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

“Enfoques en el plano: documentos y aportes para la enseñanza de Artes
Visuales en las escuelas” es una recomendación didáctica...

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-
y-guerra/)

[Efemérides: 25 de Mayo. Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revolucion-y-guerra/)

Efemérides: 25 de Mayo.<br>

**Node ID:** 23cf13d3-4189-483e-87f4-5057feffc33e<br>**Similarity:** 0.7063791122928902<br>**Text:** Aquel primer
festejo](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

25 de mayo: aquel primer festejo es una secuencia didáctica de Tu Escuela en
Casa destinada a las salas de 4 y 5 años de...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-historia/)

[25 de mayo: otra vuelta por la
historia](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-
historia/)

25 de mayo: otra vuelta por la historia es una secuencia didáctica de Tu
Escuela en Casa destinada a 4.º, 5.° y 6.º grado...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-ninas-y-ninos-en-1810/)

[25 de Mayo: niñas y niños en 1810](https://hacemosescuela.cba.gov.ar/25-de-
mayo-ninas-y-ninos-en-1810/)

25 de Mayo: niñas y niños en 1810 es una secuencia didáctica de Tu Escuela en
Casa destinada a 2.º y 3.º grado de nivel Primario....

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

[Recomendación didáctica: mundos
pluriversos](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

“Mundos pluriversos” es una recomendación didáctica de Hacemos Escuela que
recupera diversos materiales del repositorio digital...

[](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

[Unidades de longitud: el metro. La revolución de las
medidas](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

“Unidades de longitud: el metro.<br>

## Plug it into Query Engine


In [20]:
from llama_index.llms.gemini import Gemini

llm = Gemini()

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [22]:
qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un ayudante y debe responder a la pregunta del usuario usando el contexto provisto.\n"
    "Responda  de forma amable, detallada y extensa.\n"
    "Además responda con enlaces a los recursos en caso de que sea posible.\n"
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

In [28]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            ""
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [29]:
query_engine = RetrieverQueryEngine.from_args(retriever,
                                              llm=llm,
                                              text_qa_template=text_qa_template, 
                                              verbose=True, 
                                              callback_manager=callback_manager)
#base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [30]:
query_str = (
    "qué material hay para la educación secundaria?"
)

In [31]:
response = query_engine.query(query_str)

DEBUG:llama_index.core.indices.utils:> Top 4 nodes:
> [Node bd661889-f4fa-4f84-990f-ee7752b55bee] [Similarity score:             0.812927] Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fuer...
> [Node 70ce5d21-826e-4203-b344-4172fe4bc4bd] [Similarity score:             0.78243] **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ ...
> [Node 3bade23b-76df-4004-b9f3-cfed18c47f8c] [Similarity score:             0.778127] **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/ha...
> [Node 83cca01d-50c6-4e11-b7a5-09b3bd45c18a] [Similarity score:             0.759198] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> Top 4 nodes:
> [Node bd661889-f4fa-4f84-990f-ee7752b55bee] [Similarity score:             0.812927] Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fu

In [32]:
print(str(response))

**Materiales para la Educación Secundaria:**

El sitio web "Hacemos Escuela" ofrece una variedad de materiales educativos para la Educación Secundaria, que incluyen:

* **Secuencias didácticas:** Planes de lecciones detallados que cubren temas específicos del currículo.
[Enlace a las secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)

* **Objetos Hipermediales Interactivos (OHI):** Recursos digitales interactivos que permiten a los estudiantes explorar conceptos y habilidades de forma atractiva.
[Enlace a los OHI](https://hacemosescuela.cba.gov.ar/ohi)

* **Colecciones didácticas:** Conjuntos de recursos sobre temas específicos, como la alfabetización digital o la educación ambiental.
[Enlace a las colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)

* **Recomendaciones:** Sugerencias de recursos y estrategias de enseñanza para apoyar el aprendizaje de los estudiantes de secundaria.
[Enlace a las recomendaciones](https://hacemo

## Probando Sub Questions

In [28]:
import nest_asyncio 
nest_asyncio.apply()

In [30]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="sitios web",
            description="Sitios del Instituto Superior de Estudios Pedagógicos",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    llm=llm,
    query_engine_tools=query_engine_tools,
    use_async=True,
)


In [31]:
response = query_engine.query(
    "qué material hay para la educación secundaria?"
)

Generated 1 sub questions.
[sitios web] Q: Sitios web del Instituto Superior de Estudios Pedagógicos
Generated 1 sub questions.
[sitios web] Q: Sitios web del Instituto Superior de Estudios Pedagógicos
DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.742761] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 4aaf1a2a-dc10-4734-b7ce-c41fa46e9844] [Similarity score:             0.742271] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.741616] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.736671] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas]

In [33]:
print(str(response))

The provided context does not mention anything about materials for secondary education, so I cannot answer this question from the provided context.


## Debugging

In [61]:
llama_debug.get_event_pairs()

[[CBEvent(event_type=<CBEventType.NODE_PARSING: 'node_parsing'>, payload={<EventPayload.DOCUMENTS: 'documents'>: [Document(id_='https://hacemosescuela.cba.gov.ar/materiales-educativos/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](https://hacemosescuela.

In [62]:
llama_debug.get_llm_inputs_outputs()

[[CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas]

In [63]:
event_pairs = llama_debug.get_llm_inputs_outputs()

print(event_pairs[0][0])
print(event_pairs[0][1].payload.keys())
print(event_pairs[0][1].payload["response"])

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](h

In [64]:
print(event_pairs[0][0])
print(event_pairs[0][0].payload.keys())


CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](h

In [65]:
print(event_pairs[0][0].payload["messages"][1])

user: Contexto:
---------------------
[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacem

In [66]:
# Clear the currently cached events
llama_debug.flush_event_logs()

# Probando otra estrategia para retrieve

## Setup Embedding + LLM

In [3]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [4]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="this is a document"
)

In [5]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model_name="models/gemini-1.0-pro", temperature=0.3)

## Setup index

In [32]:
Settings.chunk_size = 1800
Settings.chunk_overlap = 500

In [51]:
## Pruebo un parser para markdown

from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser.from_defaults(include_metadata=True,
                                               include_prev_next_rel=True)

nodes = node_parser.get_nodes_from_documents(documents)

In [19]:
len(nodes)

310

In [52]:
# Procemos los nodos para obtener mejor información

# Dejamos afuera nodos que sean "excesivamente" grandes (límite en el servicio de google para el embedding)
nodes_small = [n for n in nodes if len(n.get_content()) < 10000]

In [53]:
# Sólo nos quedamos con nodos que tengan metadata interesante (headers, subheaders, etc)
nodes_small2 = [n for n in nodes_small if len(n.metadata) > 1]

In [48]:
# Agregamos información de metadata  cada nodo! (TEST)
for n in nodes_small2:
    n.set_content(n.get_metadata_str() + '\n' + n.get_content())


In [42]:
print(len(nodes))
print(len(nodes_small))
print(len(nodes_small2))

310
309
273


In [50]:
print(nodes_small[102].get_content())

Header_2: Educación Primaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
Educación Primaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica. Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-
en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

“Enfoques en el plano: documentos y aportes para la enseñanza de Artes
Visuales en las escuelas” es una recomendación didáctica...

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-traba

In [9]:
from llama_index.core import VectorStoreIndex

# 6 minutos aprox para 1000 nodos
sentence_index = VectorStoreIndex(nodes)

In [10]:
from llama_index.core import PromptTemplate


qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un asistente y debe responder a la pregunta del usuario usando el contexto provisto."
    "Responda de forma amable y detallada, proporcionando toda la información necesaria."
    "Si es posible organice y estructure la información mediante inferencia sobre el contexto provisto."
    "Además responda con enlaces a los recursos en caso de que sea posible."
    "Si no encuentra la respuesta en el contexto pero el mismo contiene información y enlaces relacionados a la pregunta, responda con esa información."
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

qa_template = PromptTemplate(qa_prompt_str)

In [13]:

from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.response.notebook_utils import display_response


query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
    text_qa_template=qa_template
)
window_response = query_engine.query(
    "qué material erucativo hay para la educación primaria?"
)
display_response(window_response)

**`Final Response:`** **Materiales educativos para Educación Primaria:**

* **Secuencias didácticas:** https://hacemosescuela.cba.gov.ar/materiales-educativos
* **Objetos Hipermediales Interactivos (OHI):** https://hacemosescuela.cba.gov.ar/ohi
* **Colecciones didácticas:** https://hacemosescuela.cba.gov.ar/colecciones-didacticas
* **Recomendaciones:** https://hacemosescuela.cba.gov.ar/recomendaciones

In [14]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: [ ! [Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_3-1/img/logo_202403.png?v=20240523-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversa

### Probando sub chunks!

In [151]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

sub_chunk_sizes = [500, 1000, 1500]
sub_node_parsers = [SentenceSplitter(chunk_size=c) for c in sub_chunk_sizes]

all_nodes = []

for base_node in nodes_small2:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [152]:
# Agregamos información de metadata  cada nodo! (TEST)
for n in all_nodes:
    n.set_content(n.get_metadata_str() + '\n' + n.get_content())

In [153]:
len(all_nodes)

1180

In [154]:
print(all_nodes[275].get_content())

Header_2: Entre Lenguas / Jornada extendida
Header_3: **Cuarto Grado -** Nuestra tribu / Our tribe
Header_6: **¡Seguinos!**
Source: https://hacemosescuela.cba.gov.ar/entre-lenguas/nuestra-tribu-our-tribe/
**¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_3-1/img/licencias.png)

![Logo Ministerio de Educación](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_3-1/img/Logo_MIN.png)![Logo Gobierno de
Córdoba](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_3-1/img/Logo_GOB.png)


In [155]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [ ]:
all_nodes_dict

In [127]:
vector_index_chunk = VectorStoreIndex(all_nodes,
                                      embed_model=embed_model)

In [157]:
base_index = VectorStoreIndex(
    nodes=all_nodes,
    embed_model=embed_model,
    storage_context=storage_context)

In [146]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=6)

In [147]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True
)

In [150]:
from llama_index.core.response.notebook_utils import display_source_node

nodes = retriever_chunk.retrieve(
    "Necesito material educativo para el nivel primario"
)
for node in nodes:
    #display_source_node(node, source_length=2000)
    display_source_node(node, source_length=2000)

Retrieving with query id None: Necesito material educativo para el nivel primario
Retrieved node with id, entering: 21ee864e-b39a-41ef-9f1d-8eec0229e9be
Retrieving with query id 21ee864e-b39a-41ef-9f1d-8eec0229e9be: Necesito material educativo para el nivel primario
Retrieved node with id, entering: 3bbf86a5-4557-4526-9bcc-78027aaa0eba
Retrieving with query id 3bbf86a5-4557-4526-9bcc-78027aaa0eba: Necesito material educativo para el nivel primario


**Node ID:** 21ee864e-b39a-41ef-9f1d-8eec0229e9be<br>**Similarity:** 0.8370962482196547<br>**Text:** Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.<br>

**Node ID:** 3bbf86a5-4557-4526-9bcc-78027aaa0eba<br>**Similarity:** 0.8117648634934073<br>**Text:** Header_3: **Educación permanente de jóvenes y adultos**
Header_4: Nivel Primario
Source: https://hacemosescuela.cba.gov.ar/jovenes-y-adultos/
Nivel Primario<br>

In [131]:
query_engine_chunk = RetrieverQueryEngine.from_args(retriever_chunk, 
                                                    llm=llm, 
                                                    text_qa_template=qa_template)

In [136]:
response = query_engine_chunk.query(
    "Qué material educativo hay para el nivel secundario?"
)


Retrieving with query id None: Qué material educativo hay para el nivel secundario?
Retrieved node with id, entering: 21ee864e-b39a-41ef-9f1d-8eec0229e9be
Retrieving with query id 21ee864e-b39a-41ef-9f1d-8eec0229e9be: Qué material educativo hay para el nivel secundario?
Retrieved node with id, entering: 695d8851-770f-4754-8607-b4d7ac887a44
Retrieving with query id 695d8851-770f-4754-8607-b4d7ac887a44: Qué material educativo hay para el nivel secundario?


In [137]:
display_response(response)

**`Final Response:`** El contexto proporcionado no contiene información específica sobre materiales educativos para el nivel secundario. Sin embargo, proporciona un enlace a la página web "Educación permanente de jóvenes y adultos" que puede contener información relevante:

https://hacemosescuela.cba.gov.ar/jovenes-y-adultos/

In [166]:
query_engine = base_index.as_query_engine(
    llm=llm,
    similarity_top_k=6,
    sparse_top_k=12,
    vector_store_query_mode="hybrid",
    text_qa_template=qa_template
)

In [168]:
response = query_engine.query(
    "qué material educativo hay para el nivel secundario?"
)

display_response(response)

**`Final Response:`** **Materiales educativos para el nivel secundario:**

* **La revolución de las medidas:** Secuencia didáctica para 1.° y 2.° año.
[Enlace](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-parte-ii/)

* **La revolución de los microorganismos (Parte II):** Secuencia didáctica para 1.° y 2.° año.
[Enlace](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-parte-ii/)

Para ver más materiales educativos para el nivel secundario, puedes visitar el siguiente enlace:
[Enlace](http://hacemosescuela.cba.gov.ar/buscador/?nivel=165&key=)

# Otra estrategia: Routing

In [ ]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.readers.web import SitemapReader

#sitemap="https://hacemosescuela.cba.gov.ar/post-sitemap.xml"
sitemap="https://hacemosescuela.cba.gov.ar/page-sitemap.xml"

documents = SitemapReader(html_to_text=True, limit=10).load_data(
    sitemap_url=sitemap
)

error fetching page from https://hacemosescuela.cba.gov.ar/secuencias/


In [3]:
print(len(documents))

39


## Debugging

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
## Configuración para debugging

from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [6]:
from llama_index.core import Settings

Settings.callback_manager = callback_manager

## Node parser (markdown)

In [7]:
from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser.from_defaults(include_metadata=True,
                                               include_prev_next_rel=True)

nodes = node_parser.get_nodes_from_documents(documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Programas

[ ![entre-lenguas](https://hacemoses...
> Adding chunk: Programas

[ ![entre-lenguas](https://hacemoses...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Otros sitios de interés

[ ![boton-rdu](https:/...
> Adding chunk: Otros sitios de interés

[ ![boton-rdu](https:/...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: **hacemosescuela@isep-cba.edu.ar**

[ ![](https...
> Adding chunk: **hacemosescuela@isep-cba.edu.ar**

[ ![](https...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: **¡Seguinos!**

](https://www.instagram.com/Hac...
> Adding chunk: **¡Seguinos!**

](https://www.instagram.com/Hac...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.g

In [8]:
print(len(nodes))

316


In [9]:
nodes_small = [n for n in nodes if len(n.get_content()) < 10000]

In [10]:
nodes_small2 = [n for n in nodes_small if len(n.metadata) > 1]

In [11]:
# Agregamos información de metadata  cada nodo! (TEST)
for n in nodes_small2:
    n.set_content(n.get_metadata_str() + '\n' + n.get_content())

In [12]:
print(len(nodes_small2))

277


In [13]:
print(nodes_small2[90].get_content())

Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/la-computadora-una-revolucion-en-dos-
partes-introduccion-al-software-y-hardware-libre/)

[La computadora: una revolución en dos partes. Introducción al software y
hardware libre](https://hacemosescuela.cba.gov.ar/la-computadora-una-
revolucion-en-dos-partes-introduccion-al-software-y-hardware-libre/)

“La computadora: una revolución en dos partes. Introducción al software y
hardware libre” es una secuencia didáctica de Hacemos...

[](https://hacemosescuela.cba.gov.ar/revolucion-en-tres-tiempos-cambios-y-
continuidades-en-los-dispositivos-computacionales/)

[Revolución en tres tiempos: cambios y continuidades en los dispositivos
computacionales](https://hacemosescuela.cba.gov.ar/revolucion-en-tres-tiempos-
cambios-y-continuidades-en-los-dispositivos-computacionales/)

“Revolución en tres tiempos: cambios y continuidades en los dispos

## Retriever

In [14]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [15]:
## PROBAMOS BASE DE DATOS VECTORIAL: QDRANT!
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

# creates a persistant index to disk
#client = QdrantClient(host="localhost", port=6333)
#aclient = AsyncQdrantClient(host="localhost", port=6333)

client = QdrantClient(":memory:")

# create our vector store with hybrid indexing enabled
# batch_size controls how many nodes are encoded with sparse vectors at once
vector_store = QdrantVectorStore(
    "hacemos_escuela_isep",
    client=client,
    #aclient=aclient,
    enable_hybrid=True,
    batch_size=12,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293
https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
https://huggingface.co:443 "GET /api/models/Qdrant/SPLADE_PP_en_v1/revision/main HTTP/1.1" 307 83
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293
https://huggingface.co:443 "GET /api/models/Qdrant/Splade_PP_en_v1/revision/main HTTP/1.1" 200 1293


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
# Seteamos algunos parámetros

Settings.chunk_size = 1500
Settings.chunk_overlap = 300

In [18]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY
)

## Index

In [19]:
# demora 7 min aprox para 300 nodos 
from llama_index.core import VectorStoreIndex


base_index = VectorStoreIndex(
    nodes=nodes_small2,
    embed_model=embed_model,
    storage_context=storage_context)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 7.313961 seconds
    |_CBEventType.EMBEDDING -> 4.189375 seconds
    |_CBEventType.EMBEDDING -> 3.306621 seconds
    |_CBEventType.EMBEDDING -> 3.69538 seconds
    |_CBEventType.EMBEDDING -> 3.352456 seconds
    |_CBEventType.EMBEDDING -> 3.333042 seconds
    |_CBEventType.EMBEDDING -> 3.28186 seconds
    |_CBEventType.EMBEDDING -> 3.213283 seconds
    |_CBEventType.EMBEDDING -> 3.294352 seconds
    |_CBEventType.EMBEDDING -> 3.243413 seconds
    |_CBEventType.EMBEDDING -> 3.256788 seconds
    |_CBEventType.EMBEDDING -> 3.188071 seconds
    |_CBEventType.EMBEDDING -> 3.210183 seconds
    |_CBEventType.EMBEDDING -> 3.219853 seconds
    |_CBEventType.EMBEDDING -> 3.195392 seconds
    |_CBEventType.EMBEDDING -> 11.508049 seconds
    |_CBEventType.EMBEDDING -> 5.537002 seconds
    |_CBEventType.EMBEDDING -> 5.070462 seconds
    |_CBEventType.EMBEDDING -> 4.837884 seconds
    |_CBEventType.EMBEDDING -> 5.030581 seconds
    

In [20]:
# Almacenamos index localmente para evitar que se procese nuevamente!
base_index.storage_context.persist(persist_dir="data/")

DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/docstore.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/docstore.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/index_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/index_store.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/graph_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/graph_store.json
DEBUG:fsspec.local:open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/image__vector_store.json
open file: c:/Users/Illak/proyectos/RAGs/proyecto_web_RAG/web_RAG_pruebas/data/image__vector_store.json


In [ ]:
# Cargamos el index!!

from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context_reb = StorageContext.from_defaults(persist_dir="data/")

# load index
index_reb = load_index_from_storage(storage_context,
                                embed_model=embed_model)

In [21]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model_name="models/gemini-1.0-pro", temperature=0.3)

In [22]:
from llama_index.core import PromptTemplate


qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un asistente y debe responder a la pregunta del usuario usando el contexto provisto."
    "Responda de forma amable y detallada, proporcionando toda la información necesaria."
    "Si es posible organice y estructure la información mediante inferencia sobre el contexto provisto."
    "Además responda con enlaces a los recursos en caso de que sea posible."
    "Si no encuentra la respuesta en el contexto pero el mismo contiene información y enlaces relacionados a la pregunta, responda con esa información."
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

qa_template = PromptTemplate(qa_prompt_str)


In [ ]:
retriever = base_index.as_retriever(similarity_top_k=5)

nodes_with_scores = retriever.retrieve("Qué material educativo hay para educación inicial?")

for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

## Query Engine

In [264]:
index_sel = base_index

query_engine = index_sel.as_query_engine(
    llm=llm,
    similarity_top_k=5,
    sparse_top_k=10,
    vector_store_query_mode="hybrid",
    text_qa_template=qa_template,
    use_async=True
)

In [265]:
from llama_index.core.response.notebook_utils import display_response

response = query_engine.query(
    "qué material educativo hay para la educación secundaria?"
)

display_response(response)

DEBUG:google.api_core.retry:Retrying due to 503 Connection timed out, sleeping 0.7s ...
Retrying due to 503 Connection timed out, sleeping 0.7s ...
DEBUG:llama_index.core.indices.utils:> Top 5 nodes:
> [Node 20297a38-4dc8-448c-8bd9-8eec56368dbc] [Similarity score:             0.584661] Header_2: Educación Secundaria
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacem...
> [Node 6a465a5d-6d82-4ba3-a93b-a766ec2eae0a] [Similarity score:             0.5] Header_2: Educación Secundaria
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
E...
> [Node 68999d42-b6ef-448a-be8b-4db45bb79698] [Similarity score:             0.426193] Header_3: **Educación permanente de jóvenes y adultos**
Header_4: Nivel Secundario
Source: https:...
> [Node 610e374e-12e7-416f-97c9-09bfa0f84922] [Similarity score:             0.410075] Header_2: Educación Secundaria
Header_6: **¡Seguinos!**
Source: https://hacemosescuela.cba.gov.ar...
> [Node 52bf5c57-2142-4ec3-872f-ccd0bdb5f849] [Simila

**`Final Response:`** **Materiales Educativos para Educación Secundaria:**

* **La computadora: una revolución en dos partes. Introducción al software y hardware libre:** https://hacemosescuela.cba.gov.ar/la-computadora-una-revolucion-en-dos-partes-introduccion-al-software-y-hardware-libre/
* **Revolución en tres tiempos: cambios y continuidades en los dispositivos computacionales:** https://hacemosescuela.cba.gov.ar/revolucion-en-tres-tiempos-cambios-y-continuidades-en-los-dispositivos-computacionales/
* **Revoluciones del yo: ficciones autobiográficas:** https://hacemosescuela.cba.gov.ar/revoluciones-del-yo-ficciones-autobiograficas/
* **¡Música y revoluciones! Un breve recorrido por América Latina:** https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-un-breve-recorrido-por-america-latina/
* **¡Música y revoluciones! Ritornelo de la memoria histórica:** https://hacemosescuela.cba.gov.ar/musica-y-revoluciones-ritornelo-de-la-memoria-historica/
* **Recomendación didáctica. Enfoques en el plano: documentos y aportes para la enseñanza de Artes Visuales en las escuelas:** https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-escuelas/
* **Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos, reivindicaciones y luchas:** https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/
* **Efemérides: 25 de Mayo. Patria, revolución y guerra:** https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-y-guerra/
* **25 de Mayo: protagonistas de una revolución:** https://hacemosescuela.cba.gov.ar/25-de-mayo-protagonistas-de-una-revolucion/
* **La revolución de las lenguas y los lectos:** https://hacemosescuela.cba.gov.ar/la-revolucion-de-las-lenguas-y-los-lectos-2/
* **La probabilidad. La revolución de los datos:** https://hacemosescuela.cba.gov.ar/la-probabilidad-la-revolucion-de-los-datos/
* **Recomendación didáctica: mundos pluriversos:** https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-pluriversos/

## Cargamos documento de preguntas frecuentes

In [60]:
from pathlib import Path
from llama_index.readers.file import PDFReader, PyMuPDFReader


loader = PyMuPDFReader()
docs0 = loader.load_data(file_path=Path("faq_mesa_ayuda.pdf"))

In [205]:
from llama_index.readers.web import SimpleWebPageReader

documents_html = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://docs.google.com/document/d/e/2PACX-1vQXmLg64DYEo4h_JyLSFM1ci7pY_VhrmwTFATcqR6MpcjB4ZK5w5BnTSAiJLQ6kMh-7uSV_ZfTXpqmp/pub"]
)

documents_html[0]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): docs.google.com:443
Starting new HTTPS connection (1): docs.google.com:443
DEBUG:urllib3.connectionpool:https://docs.google.com:443 "GET /document/d/e/2PACX-1vQXmLg64DYEo4h_JyLSFM1ci7pY_VhrmwTFATcqR6MpcjB4ZK5w5BnTSAiJLQ6kMh-7uSV_ZfTXpqmp/pub HTTP/1.1" 200 None
https://docs.google.com:443 "GET /document/d/e/2PACX-1vQXmLg64DYEo4h_JyLSFM1ci7pY_VhrmwTFATcqR6MpcjB4ZK5w5BnTSAiJLQ6kMh-7uSV_ZfTXpqmp/pub HTTP/1.1" 200 None


Document(id_='https://docs.google.com/document/d/e/2PACX-1vQXmLg64DYEo4h_JyLSFM1ci7pY_VhrmwTFATcqR6MpcjB4ZK5w5BnTSAiJLQ6kMh-7uSV_ZfTXpqmp/pub', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Se publicó con Documentos de Google\n\n[Denunciar\nabuso](//drive.google.com/abuse?id=AKkXjoxXr5uuv6BXR_WNK4WkPnKq29U3gSXcsmKpXTQIw2KLR0ssnxmuoUkjvqYpPjr9I3z9YX0O6dQee1OAkhQ:0)[Más\ninformación](https://support.google.com/docs/answer/183965 "Learn more")\n\nCopia Respuestas frecuentes | Mesa |\n\nSe actualiza automáticamente cada 5 minutos\n\nRESPUESTAS FRECUENTES\n\n# GENERALES  \n\n## Orientaciones para el acceso a Autogestión y Campus Virtual\n\nHola NOMBRE, buenas tardes.\n\nSi usted ya posee una cuenta de CiDi nivel 2, cuando ingrese al portal y haga\nclic en Iniciar Sesión, el sistema se redirigirá a la página del Ciudadano\nDigital. Allí deberá completar los campos de CUIL y contraseña para que el\nsistema lo identifique y

In [206]:
print(documents_html[0].get_content())

Se publicó con Documentos de Google

[Denunciar
abuso](//drive.google.com/abuse?id=AKkXjoxXr5uuv6BXR_WNK4WkPnKq29U3gSXcsmKpXTQIw2KLR0ssnxmuoUkjvqYpPjr9I3z9YX0O6dQee1OAkhQ:0)[Más
información](https://support.google.com/docs/answer/183965 "Learn more")

Copia Respuestas frecuentes | Mesa |

Se actualiza automáticamente cada 5 minutos

RESPUESTAS FRECUENTES

# GENERALES  

## Orientaciones para el acceso a Autogestión y Campus Virtual

Hola NOMBRE, buenas tardes.

Si usted ya posee una cuenta de CiDi nivel 2, cuando ingrese al portal y haga
clic en Iniciar Sesión, el sistema se redirigirá a la página del Ciudadano
Digital. Allí deberá completar los campos de CUIL y contraseña para que el
sistema lo identifique y lo redirija otra vez al portal del ISEP. Puede
conocer en detalle cómo es el procedimiento de logueo e ingreso a través del
siguiente tutorial: [clic
aquí](https://www.google.com/url?q=https://docs.google.com/document/d/1fBhdS_oZ3FYGkUBA4Jt_njY3oZKJBas570M2UKiAnFY/edit?usp%3Dshari

In [207]:
## Pruebo un parser para markdown

from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser.from_defaults(include_metadata=True,
                                               include_prev_next_rel=True)

nodes_docs = node_parser.get_nodes_from_documents(documents_html)

# Agregamos información de metadata  cada nodo! (TEST)
for n in nodes_docs:
    n.set_content(n.get_metadata_str() + '\n' + n.get_content())

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Se publicó con Documentos de Google

[Denunciar...
> Adding chunk: Se publicó con Documentos de Google

[Denunciar...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: GENERALES
> Adding chunk: GENERALES
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Orientaciones para el acceso a Autogestión y Ca...
> Adding chunk: Orientaciones para el acceso a Autogestión y Ca...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Capacitaciones vigentes: Me quiero inscribir en...
> Adding chunk: Capacitaciones vigentes: Me quiero inscribir en...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Modalidad de cursado

Caso 1: Consulta general ...
> Adding chunk: Modalidad de cursado

Caso 1: Consulta general ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Inscripciones: ¿Cómo me inscribo a una capacita...
> Adding chunk: Inscripciones: ¿Cómo me inscribo a una capacita...
DE

In [208]:
print(len(nodes_docs))
print(nodes_docs[57].get_content())

59
Header_1: DATOS DE CONTACTO IFD (Instituto de Formación Docente)
DATOS DE CONTACTO IFD (Instituto de Formación Docente)

Instituto de Enseñanza Superior Simón Bolívar

E-mail: iessb@isep-cba.edu.ar

CPI: Francisco Viale

CPI: Ana Ludueña

Instituto Superior “Carlos Alberto Leguizamón”

E-mail: iscal@isep-cba.edu.ar

CPI: Claudia Kühn

Escuela Normal Superior “Dr. Alejandro Carbó”

E-mail: enscarbo@isep-cba.edu.ar

CPI: Laura Cecilia López

Escuela Normal Superior “Dr. Agustín Garzón Agulla” – Córdoba

E-mail: ensagulla@isep-cba.edu.ar

CPI: María Delia Puebla Valente

Instituto Superior Dr. Carlos María Carena

E-mail: carena@isep-cba.edu.ar

CPI: María Eugenia Allende

Escuela Normal Superior Justo José de Urquiza

E-mail: [urquiza@isep-cba.edu.ar](mailto:urquiza@isep-cba.edu.ar)

CPI: Fernando Ezequiel Dehaes

CPI: Paola Andrea Toranzo

Instituto de Educación Superior del Centro de la República Argentina Dr. Ángel
Diego Márquez INESCER

E-mail: inescer@isep-cba.edu.ar

CPI: Paola 

In [187]:
qa_prompt_str_faq  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un asistente de mesa de ayuda/entrada y debe responder a la pregunta del usuario usando el contexto provisto."
    "Responda de forma amable y detallada, proporcionando toda la información necesaria incluido enlaces."
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

qa_faq_template = PromptTemplate(qa_prompt_str_faq)

In [249]:

# define vector retriever
vector_index = VectorStoreIndex(nodes=nodes_docs, 
                                embed_model=embed_model,)

vector_query_engine = vector_index.as_query_engine(similarity_top_k=6, 
                                                   llm=llm,
                                                   use_async=True,
                                                   text_qa_template=qa_faq_template)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 3.413922 seconds
    |_CBEventType.EMBEDDING -> 3.257645 seconds
    |_CBEventType.EMBEDDING -> 3.346285 seconds
    |_CBEventType.EMBEDDING -> 4.47795 seconds
    |_CBEventType.EMBEDDING -> 3.227762 seconds
    |_CBEventType.EMBEDDING -> 3.831998 seconds
**********


In [252]:
res = vector_query_engine.query("Necesito información sobre cómo acceder al campus")

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node ab850cda-6cc6-42f1-a1c3-f6ae1da3c9df] [Similarity score:             0.926959] Header_1: GENERALES  
Header_2: Acceso al campus y a las aulas
Header_5: Para acceder al Campus v...
> [Node 4fabae8c-3185-43bc-bb91-8243b7e83e16] [Similarity score:             0.893381] Header_1: GENERALES  
Header_2: Acceso al campus y a las aulas
Header_5: Hola NOMBRE, buenos días...
> [Node 602e39b9-b6d6-4b3e-81d6-aacd27b9e33c] [Similarity score:             0.885255] Header_1: GENERALES  
Header_2: Acceso al campus y a las aulas
Header_5: En principio, deberá ing...
> [Node aa6021c3-0133-4db7-ace5-2927f1ee5645] [Similarity score:             0.840343] Header_1: GENERALES  
Header_2: Acceso al campus y a las aulas
Header_5: Para continuar, tendrá q...
> [Node 58451d03-4092-498b-96b5-3b1be4c77342] [Similarity score:             0.836519] Header_1: POR TRAYECTO  
POR TRAYECTO
> [Node 50640929-6001-4932-8377-5b5f556487f4] [Similarity score:       

In [253]:
display_response(res)

**`Final Response:`** Para acceder al Campus virtual, siga estos pasos:

1. Ingrese a su cuenta de Gmail institucional (x) y cierre su cuenta personal si está abierta. También puede acceder a su Gmail desde una ventana de incógnito para asegurarse de que su cuenta personal no esté abierta.
2. Abra la página del ISEP y acceda mediante la plataforma Ciudadano Digital con su número de CUIL y contraseña.
3. Desde esa plataforma, será redirigido al Portal del ISEP desde donde podrá ingresar al Campus virtual.

In [279]:
from llama_index.core.tools import QueryEngineTool

hacemos_escuela_tool = QueryEngineTool.from_defaults(
    name="Hacemos Escuela",
    query_engine=query_engine,
    description=(
        'Esta herramienta responde preguntas específicas sobre material educativo en el sitio "Hacemos Escuela".'
        'Preguntas que refieren a la búsqueda sobre material educativo y/o material didáctico.')
)

faq_tool = QueryEngineTool.from_defaults(
    name="FAQ",
    query_engine=vector_query_engine,
    description=(
        'Esta herramienta responde preguntas frecuentes sobre ISEP. Son preguntas que responde mesa de ayuda.'
        'Son extraídas a partir de un documento de preguntas frecuentes'
    ),
)

In [280]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    [hacemos_escuela_tool, faq_tool],
    index_cls=VectorStoreIndex, embed_model=embed_model
)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 0.650339 seconds
**********


In [281]:
from llama_index.core.query_engine import ToolRetrieverRouterQueryEngine

query_engine_final = ToolRetrieverRouterQueryEngine(obj_index.as_retriever(), llm=llm)

In [282]:
new__tmpl_str = (
    "El contexto con información proveniente de diversas fuentes es el siguiente:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "A partir de la información proveniente de múltiples fuentes y sin conocimiento previo, responda la pregunta.\n"
    "Pregunta: {query_str}\n"
    "Respuesta: "
)
new_tmpl = PromptTemplate(new__tmpl_str)

query_engine_final.update_prompts(
    {"summarizer:summary_template": new_tmpl}
)

In [283]:
response = query_engine_final.query("qué material educativo hay para la educación inicial?")

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node -3687948936918193558] [Similarity score:             0.765009] Tool name: Hacemos Escuela
Tool description: Esta herramienta responde preguntas específicas sobr...
> [Node 4886553331935436775] [Similarity score:             0.668749] Tool name: FAQ
Tool description: Esta herramienta responde preguntas frecuentes sobre ISEP. Son p...
> Top 2 nodes:
> [Node -3687948936918193558] [Similarity score:             0.765009] Tool name: Hacemos Escuela
Tool description: Esta herramienta responde preguntas específicas sobr...
> [Node 4886553331935436775] [Similarity score:             0.668749] Tool name: FAQ
Tool description: Esta herramienta responde preguntas frecuentes sobre ISEP. Son p...
DEBUG:llama_index.core.indices.utils:> Top 5 nodes:
> [Node 76a52edd-39c3-4e9f-9391-b01497f26b91] [Similarity score:             0.598851] Header_2: Materiales educativos
Source: https://hacemosescuela.cba.gov.ar/materiales-educativos/
...
> [Node

In [284]:
display_response(response)

**`Final Response:`** * **Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos, reivindicaciones y luchas:** https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/

* **25 de mayo: aquel primer festejo:** https://hacemosescuela.cba.gov.ar/25-de-mayo-aquel-primer-festejo-4/

* **La enseñanza de los números a lo largo de la escolaridad:** https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-lo-largo-de-la-escolaridad/

* **La enseñanza de la descripción a lo largo de la escolaridad:** https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-a-lo-largo-de-la-escolaridad/

* **Miradas pixeladas:** https://hacemosescuela.cba.gov.ar/miradas-pixeladas/

* **Te invito a mi jardín. Conocer y describir nuestro espacio de trabajo:** https://hacemosescuela.cba.gov.ar/te-invito-a-mi-jardin-conocer-y-describir-nuestro-espacio-de-trabajo/

* **Un recorrido por el jardín (Parte III):** https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-iii/

* **Un recorrido por el jardín (Parte II):** https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-ii/

* **Un recorrido por el jardín (Parte I):** https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-i/

* **En la red, pero sin enredarme (Parte II):** https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/

* **En la red, pero sin enredarme (Parte I):** https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/

* **Había una vez… ¡te lo cuento otra vez!:** https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/

* **Jugamos con el nombre:** https://drive.google.com/file/d/1psc3ufFcznJrChIPOC1-66LbweWJvVtI/view?usp=sharing

* **Un barco muy pirata:** https://drive.google.com/file/d/1Q_f5tXPD-yy20pbq7Ado42aMd9EBMbRw/view

In [287]:
response = query_engine_final.query("Necesito información para comunicarme con el Simón Bolivar")
display_response(response)

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node -3687948936918193558] [Similarity score:             0.675887] Tool name: Hacemos Escuela
Tool description: Esta herramienta responde preguntas específicas sobr...
> [Node 4886553331935436775] [Similarity score:             0.660199] Tool name: FAQ
Tool description: Esta herramienta responde preguntas frecuentes sobre ISEP. Son p...
> Top 2 nodes:
> [Node -3687948936918193558] [Similarity score:             0.675887] Tool name: Hacemos Escuela
Tool description: Esta herramienta responde preguntas específicas sobr...
> [Node 4886553331935436775] [Similarity score:             0.660199] Tool name: FAQ
Tool description: Esta herramienta responde preguntas frecuentes sobre ISEP. Son p...
DEBUG:llama_index.core.indices.utils:> Top 5 nodes:
> [Node 02bae1d2-fd7f-42a9-84e4-8c142405e34f] [Similarity score:             0.5] Header_3: Contacto
Header_6: **hacemosescuela@isep-cba.edu.ar**
Source: https://hacemosescuela.cb...
> [Node b7f1

**`Final Response:`** E-mail: [iessb@isep-cba.edu.ar](mailto:iessb@isep-cba.edu.ar)

In [273]:
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

prompts_dict = query_engine_final.get_prompts()
display_prompt_dict(prompts_dict)


**Prompt Key**: summarizer:summary_template<br>**Text:** <br>

El contexto con información proveniente de diversas fuentes es el siguiente:
---------------------
{context_str}
---------------------
A partir de la información proveniente de múltiples fuentes y sin conocimiento previo, responda la pregunta.Pregunta: {query_str}
Respuesta: 


<br><br>